In [1]:
import numpy as np

In [2]:
# Загрузить и подготовить тренировочные данные из формата CSV в список
training_data = open("dataset/Data_train.csv", 'r') # 'r' - открываем файл для чтения
training_data_list = training_data.readlines() # readlines() - читает все строки в файле в переменную training_data_list
training_data.close() # закрываем файл csv

# Загрузить и подготовить тестовые данные из формата CSV в список
test_data = open("dataset/Data_test.csv", 'r') # 'r' - открываем файл для чтения
test_data_list = test_data.readlines()# Загрузить и подготовить тестовые данные из формата CSV в список 
test_data.close() # закрываем файл csv

In [3]:
# Определение класса нейронной сети
class neuron_Net:
    
    # Инициализация весов нейронной сети
    def __init__(self, input_num, neuron_num, learningrate): #констр.(кол-во входов, кол-во нейронов, скорость обучения)
                                         # МАТРИЦА ВЕСОВ
        # Задаем матрицу весов как случайное от -0,5 до 0,5
        self.weights = (np.random.rand(neuron_num, input_num) -0.5) 
        # Задаем параметр скорости обучения
        self.lr = learningrate
        
        pass
    
    # Метод обучения нейронной сети
    def train(self, inputs_list, targets_list): # принимает (вх. список данных, ответы)
        # Преобразовать список входов в вертикальный массив. .T - транспонирование
        inputs_x = np.array(inputs_list, ndmin=2).T # матрица числа
        targets_Y = np.array(targets_list, ndmin=2).T # матрица ответов: какое это число
        
                                           # ВЫЧИСЛЕНИЕ СИГНАЛОВ
        # Вычислить сигналы в нейронах. Взвешенная сумма.
        x = np.dot(self.weights, inputs_x) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y = 1/(1+np.exp(-x))        
        
                                            # ВЫЧИСЛЕНИЕ ОШИБКИ
        #  Ошибка E = -(цель - фактическое значение) 
        E = -(targets_Y - y) 
        
                                            # ОБНОВЛЕНИЕ ВЕСОВ
        # Меняем веса по каждой связи
        self.weights -= self.lr * np.dot((E * y * (1.0 - y)), np.transpose(inputs_x))
        
        pass
    
    # Метод прогона тестовых значений
    def query(self, inputs_list): # Принимает свой набор тестовых данных
        # Преобразовать список входов в вертикальный 2D массив. 
        inputs_x = np.array(inputs_list, ndmin=2).T 
        
        # Вычислить сигналы в нейронах. Взвешенная сумма.
        x = np.dot(self.weights, inputs_x)
        # Вычислить сигналы, выходящие из нейрона. Сигмоида(x)
        y = 1/(1+np.exp(-x))
        
        return y

In [4]:
                            # ЗАДАЁМ ПАРАМЕТРЫ СЕТИ
# Количество входных данных, нейронов
data_input = 2
data_neuron = 2

# Cкорость обучения
learningrate = 0.1

# Создать экземпляр нейронной сети
n = neuron_Net(data_input, data_neuron, learningrate)

In [5]:
                            # ОБУЧЕНИЕ
# Зададим количество эпох
epochs = 10000
# Прогон по обучающей выборке
for e in range(epochs):
    for i in training_data_list:
        # Получить входные данные числа
        all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
        inputs_x = np.asfarray(all_values[1:])
        
        # Получить целевое значение Y, (ответ - какое это число)
        targets_Y = int(all_values[0])  # перевод символов в int, 0 элемент - ответ
        
        # создать целевые выходные значения (все 0.01, кроме нужной метки, которая равна 0.99)
        targets_Y = np.zeros(data_neuron) + 0.01
        
        # Получить целевое значение Y, (ответ - какое это число). all_values[0] - целевая метка для этой записи
        targets_Y[int(all_values[0])] = 0.99
            
        n.train(inputs_x, targets_Y) # наш метод train - обучение нейронной сети                      

In [6]:
# Вывод обученных весов
print('Весовые коэффициенты:\n', n.weights)

Весовые коэффициенты:
 [[ 0.0082007  0.0082007]
 [-0.0082007 -0.0082007]]


In [7]:
# Прогоним входные данные из обучающей выборки через обученную сеть
for i in training_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    inputs_x = np.asfarray(all_values[1:])
    # Прогон по сети
    outputs = n.query(inputs_x)
    print(all_values[1], 'XOR', all_values[2], outputs)

0 XOR 0
 [[ 0.5]
 [ 0.5]]
1 XOR 0
 [[ 0.50205016]
 [ 0.49794984]]
0 XOR 1
 [[ 0.50205016]
 [ 0.49794984]]
1 XOR 1
 [[ 0.50410026]
 [ 0.49589974]]
